In [111]:
import pandas as pd
import numpy as np 
import json
import psycopg2
from sqlalchemy import create_engine
from config import dbusername, dbpassword

In [112]:
csv_file = "malaria/Malaria.csv"
malaria_cases_Reported = pd.read_csv(csv_file)
malaria_cases_Reported

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,69397.0,77549.0,54840.0,39263.0,61362.0,86895.0,139087.0,161778.0,NaN,NaN
2,Angola,AGO,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,1682870.0,1632282.0,1496834.0,1999868.0,2298979.0,2769305.0,3794253.0,3874892.0,NaN,NaN
3,Albania,ALB,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Kosovo,XKX,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,"Yemen, Rep.",YEM,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,106697.0,90410.0,109908.0,102778.0,86707.0,76259.0,98701.0,114004.0,NaN,NaN
256,South Africa,ZAF,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,8060.0,9866.0,5629.0,8645.0,11705.0,4357.0,4323.0,28295.0,NaN,NaN
257,Zambia,ZMB,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4077547.0,4184661.0,4851319.0,5505639.0,NaN,NaN


In [113]:
csv_file2 = "malaria/Metadata_Country.csv"
income_group = pd.read_csv(csv_file2)
income_group

,Country Code,Region,IncomeGroup,TableName
0,ABW,Latin America & Caribbean,High income,Aruba
1,AFG,South Asia,Low income,Afghanistan
2,AGO,Sub-Saharan Africa,Lower middle income,Angola
3,ALB,Europe & Central Asia,Upper middle income,Albania
4,AND,Europe & Central Asia,High income,Andorra
...,...,...,...,...
253,XKX,Europe & Central Asia,Upper middle income,Kosovo
254,YEM,Middle East & North Africa,Low income,"Yemen, Rep."
255,ZAF,Sub-Saharan Africa,Upper middle income,South Africa
256,ZMB,Sub-Saharan Africa,Lower middle income,Zambia


In [114]:
csv_file3 = "malaria/population.csv"
population_data = pd.read_csv(csv_file3)
new_population = population_data.drop(columns=['Country Name'])
new_population

,Country Code,2010,2011,2012,2013,2014,2015,2016,2017
0,ABW,101669,102046,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0
1,AFG,29185507,30117413,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0
2,AGO,23356246,24220661,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0
3,ALB,2913021,2905195,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0
4,AND,84449,83747,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0
...,...,...,...,...,...,...,...,...,...
257,XKX,1775680,1791000,1805200.0,1824100.0,1821800.0,1801800.0,1816200.0,1830700.0
258,YEM,23154855,23807588,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,27834821.0
259,ZAF,51216964,52004172,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,57000451.0
260,ZMB,13605984,14023193,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0,16853688.0


In [115]:
new_malaria_cases_reported =malaria_cases_Reported.dropna(how='all', axis=1)
new_malaria_cases_reported

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2010,2011,2012,2013,2014,2015,2016,2017
0,Aruba,ABW,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Malaria cases reported,SH.STA.MALR,94475.0,69397.0,77549.0,54840.0,39263.0,61362.0,86895.0,139087.0,161778.0
2,Angola,AGO,Malaria cases reported,SH.STA.MALR,NaN,1682870.0,1632282.0,1496834.0,1999868.0,2298979.0,2769305.0,3794253.0,3874892.0
3,Albania,ALB,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Kosovo,XKX,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,"Yemen, Rep.",YEM,Malaria cases reported,SH.STA.MALR,1394495.0,106697.0,90410.0,109908.0,102778.0,86707.0,76259.0,98701.0,114004.0
256,South Africa,ZAF,Malaria cases reported,SH.STA.MALR,NaN,8060.0,9866.0,5629.0,8645.0,11705.0,4357.0,4323.0,28295.0
257,Zambia,ZMB,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,4077547.0,4184661.0,4851319.0,5505639.0


In [116]:
new_income_group = income_group.drop(columns=['Region','TableName'])
new_income_group

,Country Code,IncomeGroup
0,ABW,High income
1,AFG,Low income
2,AGO,Lower middle income
3,ALB,Upper middle income
4,AND,High income
...,...,...
253,XKX,Upper middle income
254,YEM,Low income
255,ZAF,Upper middle income
256,ZMB,Lower middle income


In [117]:
Joined_data = new_malaria_cases_reported.set_index('Country Code').join(new_income_group.set_index('Country Code'))
Joined_data

,Country Name,Indicator Name,Indicator Code,2000,2010,2011,2012,2013,2014,2015,2016,2017,IncomeGroup
Country Code,,,,,,,,,,,,,
ABW,Aruba,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High income
AFG,Afghanistan,Malaria cases reported,SH.STA.MALR,94475.0,69397.0,77549.0,54840.0,39263.0,61362.0,86895.0,139087.0,161778.0,Low income
AGO,Angola,Malaria cases reported,SH.STA.MALR,NaN,1682870.0,1632282.0,1496834.0,1999868.0,2298979.0,2769305.0,3794253.0,3874892.0,Lower middle income
ALB,Albania,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Upper middle income
AND,Andorra,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High income
...,...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,Kosovo,Malaria cases reported,SH.STA.MALR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Upper middle income
YEM,"Yemen, Rep.",Malaria cases reported,SH.STA.MALR,1394495.0,106697.0,90410.0,109908.0,102778.0,86707.0,76259.0,98701.0,114004.0,Low income
ZAF,South Africa,Malaria cases reported,SH.STA.MALR,NaN,8060.0,9866.0,5629.0,8645.0,11705.0,4357.0,4323.0,28295.0,Upper middle income


In [118]:
Joined_data2 = Joined_data[['Country Name', 'Indicator Name', 'IncomeGroup', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']]
Joined_data2

,Country Name,Indicator Name,IncomeGroup,2010,2011,2012,2013,2014,2015,2016,2017
Country Code,,,,,,,,,,,
ABW,Aruba,Malaria cases reported,High income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,Afghanistan,Malaria cases reported,Low income,69397.0,77549.0,54840.0,39263.0,61362.0,86895.0,139087.0,161778.0
AGO,Angola,Malaria cases reported,Lower middle income,1682870.0,1632282.0,1496834.0,1999868.0,2298979.0,2769305.0,3794253.0,3874892.0
ALB,Albania,Malaria cases reported,Upper middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AND,Andorra,Malaria cases reported,High income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
XKX,Kosovo,Malaria cases reported,Upper middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YEM,"Yemen, Rep.",Malaria cases reported,Low income,106697.0,90410.0,109908.0,102778.0,86707.0,76259.0,98701.0,114004.0
ZAF,South Africa,Malaria cases reported,Upper middle income,8060.0,9866.0,5629.0,8645.0,11705.0,4357.0,4323.0,28295.0


In [119]:
renamed_col_joined_data = Joined_data2.rename(columns={'2010':'Malaria Cases 2010', '2011':'Malaria Cases 2011', '2012':'Malaria Cases 2012', '2013':'Malaria Cases 2013', '2014':'Malaria Cases 2014', '2015':'Malaria Cases 2015', '2016':'Malaria Cases 2016', '2017':'Malaria Cases 2017'})
renamed_col_joined_data

,Country Name,Indicator Name,IncomeGroup,Malaria Cases 2010,Malaria Cases 2011,Malaria Cases 2012,Malaria Cases 2013,Malaria Cases 2014,Malaria Cases 2015,Malaria Cases 2016,Malaria Cases 2017
Country Code,,,,,,,,,,,
ABW,Aruba,Malaria cases reported,High income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,Afghanistan,Malaria cases reported,Low income,69397.0,77549.0,54840.0,39263.0,61362.0,86895.0,139087.0,161778.0
AGO,Angola,Malaria cases reported,Lower middle income,1682870.0,1632282.0,1496834.0,1999868.0,2298979.0,2769305.0,3794253.0,3874892.0
ALB,Albania,Malaria cases reported,Upper middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AND,Andorra,Malaria cases reported,High income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
XKX,Kosovo,Malaria cases reported,Upper middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YEM,"Yemen, Rep.",Malaria cases reported,Low income,106697.0,90410.0,109908.0,102778.0,86707.0,76259.0,98701.0,114004.0
ZAF,South Africa,Malaria cases reported,Upper middle income,8060.0,9866.0,5629.0,8645.0,11705.0,4357.0,4323.0,28295.0


In [120]:
sum_from_2010_2017 = renamed_col_joined_data.sum(axis=0, skipna=True)
avg_from_2010_2017 = renamed_col_joined_data.median(axis=0, skipna=True)
print(sum_from_2010_2017)
print(avg_from_2010_2017)

Country Name          ArubaAfghanistanAngolaAlbaniaAndorraArab World...
Indicator Name        Malaria cases reportedMalaria cases reportedMa...
Malaria Cases 2010                                          2.44628e+07
Malaria Cases 2011                                          2.45944e+07
Malaria Cases 2012                                          4.02608e+07
Malaria Cases 2013                                          4.76141e+07
Malaria Cases 2014                                          7.74306e+07
Malaria Cases 2015                                          8.80244e+07
Malaria Cases 2016                                          1.12276e+08
Malaria Cases 2017                                           1.1805e+08
dtype: object
Malaria Cases 2010    19680.0
Malaria Cases 2011    19739.0
Malaria Cases 2012    17403.5
Malaria Cases 2013    17128.0
Malaria Cases 2014    15874.5
Malaria Cases 2015    13151.5
Malaria Cases 2016    11944.5
Malaria Cases 2017    15198.0
dtype: float64


In [121]:
complete_Malaria_Cases = renamed_col_joined_data.merge(population_data)
complete_data = complete_Malaria_Cases.rename(columns={'2010':'Population 2010', '2011':'Population 2011', '2012':'Population 2012', '2013':'Population 2013', '2014':'Population 2014', '2015':'Population 2015', '2016':'Population 2016', '2017':'Population 2017'})
complete_data2 = complete_data.drop(columns=['Country Code'])
complete_data2

,Country Name,Indicator Name,IncomeGroup,Malaria Cases 2010,Malaria Cases 2011,Malaria Cases 2012,Malaria Cases 2013,Malaria Cases 2014,Malaria Cases 2015,Malaria Cases 2016,Malaria Cases 2017,Population 2010,Population 2011,Population 2012,Population 2013,Population 2014,Population 2015,Population 2016,Population 2017
0,Aruba,Malaria cases reported,High income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101669,102046,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0
1,Afghanistan,Malaria cases reported,Low income,69397.0,77549.0,54840.0,39263.0,61362.0,86895.0,139087.0,161778.0,29185507,30117413,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0
2,Angola,Malaria cases reported,Lower middle income,1682870.0,1632282.0,1496834.0,1999868.0,2298979.0,2769305.0,3794253.0,3874892.0,23356246,24220661,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0
3,Albania,Malaria cases reported,Upper middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2913021,2905195,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0
4,Andorra,Malaria cases reported,High income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84449,83747,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,Kosovo,Malaria cases reported,Upper middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1775680,1791000,1805200.0,1824100.0,1821800.0,1801800.0,1816200.0,1830700.0
253,"Yemen, Rep.",Malaria cases reported,Low income,106697.0,90410.0,109908.0,102778.0,86707.0,76259.0,98701.0,114004.0,23154855,23807588,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,27834821.0
254,South Africa,Malaria cases reported,Upper middle income,8060.0,9866.0,5629.0,8645.0,11705.0,4357.0,4323.0,28295.0,51216964,52004172,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,57000451.0
255,Zambia,Malaria cases reported,Lower middle income,NaN,NaN,NaN,NaN,4077547.0,4184661.0,4851319.0,5505639.0,13605984,14023193,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0,16853688.0


In [122]:
Complete_datasets = complete_data2.dropna(subset=["Malaria Cases 2010", "Malaria Cases 2011", "Malaria Cases 2012", "Malaria Cases 2013", "Malaria Cases 2014", "Malaria Cases 2015", "Malaria Cases 2016", "Malaria Cases 2017"])
Complete_datasets

,Country Name,Indicator Name,IncomeGroup,Malaria Cases 2010,Malaria Cases 2011,Malaria Cases 2012,Malaria Cases 2013,Malaria Cases 2014,Malaria Cases 2015,Malaria Cases 2016,Malaria Cases 2017,Population 2010,Population 2011,Population 2012,Population 2013,Population 2014,Population 2015,Population 2016,Population 2017
1,Afghanistan,Malaria cases reported,Low income,69397.0,77549.0,54840.0,39263.0,61362.0,86895.0,139087.0,161778.0,29185507,30117413,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0
2,Angola,Malaria cases reported,Lower middle income,1682870.0,1632282.0,1496834.0,1999868.0,2298979.0,2769305.0,3794253.0,3874892.0,23356246,24220661,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0
7,Argentina,Malaria cases reported,Upper middle income,72.0,18.0,4.0,4.0,4.0,11.0,7.0,18.0,40788453,41261490,41733271.0,42202935.0,42669500.0,43131966.0,43590368.0,44044811.0
13,Azerbaijan,Malaria cases reported,Upper middle income,52.0,8.0,4.0,4.0,2.0,1.0,1.0,1.0,9054332,9173082,9295784.0,9416801.0,9535079.0,9649341.0,9757812.0,9854033.0
14,Burundi,Malaria cases reported,Low income,1763447.0,1571874.0,2151076.0,4141387.0,4585273.0,5159706.0,8274062.0,7670177.0,8675602,8958406,9245988.0,9540289.0,9844297.0,10160030.0,10487998.0,10827024.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Vietnam,Malaria cases reported,Lower middle income,17515.0,16612.0,19638.0,17128.0,15752.0,9331.0,4161.0,6142.0,87967651,88871561,89802487.0,90753472.0,91714595.0,92677076.0,93638724.0,94596642.0
250,Vanuatu,Malaria cases reported,Lower middle income,8169.0,4820.0,3435.0,2381.0,982.0,423.0,1868.0,1072.0,236211,242653,249499.0,256635.0,263888.0,271130.0,278330.0,285510.0
253,"Yemen, Rep.",Malaria cases reported,Low income,106697.0,90410.0,109908.0,102778.0,86707.0,76259.0,98701.0,114004.0,23154855,23807588,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,27834821.0
254,South Africa,Malaria cases reported,Upper middle income,8060.0,9866.0,5629.0,8645.0,11705.0,4357.0,4323.0,28295.0,51216964,52004172,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,57000451.0


In [123]:
json_file = "malaria/reporteddeaths.json"
reported_deaths= pd.read_json(json_file)
reported_deaths.columns
json_deaths = reported_deaths.drop(columns=["Last\navailable", ""])
json_deaths

,Countries,2016.0,2015.0,2014.0,2013.0,2012.0,2011.0,2010.0
0,Albania,,,,,,,12139
1,Anguilla,91,65,59,78,68,61,65
2,Antigua and Barbuda,,528,580,471,493,,
3,Argentina,,330588,322035,323327,317144,316715,316298
4,Armenia,28226,27878,27714,27196,27600,27963,27921
...,...,...,...,...,...,...,...,...
136,Uruguay,,32967,32122,32698,33354,,32572
137,Uzbekistan,,,149761,145672,145988,143253,138408
138,Venezuela (Bolivarian Republic of),,,,149876,148087,143020,138504
139,Virgin Islands (USA),,655,,,723,711,702


In [126]:
rds_connection_string = f"{dbusername}:{dbpassword}@localhost:5432/MalariaCases"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [127]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "PostgreSQL 12"

(Background on this error at: http://sqlalche.me/e/e3q8)